In [92]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from collections import Counter

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="971d355e612f4cb195f9d2314f0134cb",
                                               client_secret="d85c6e4d353447b1a5455fe011eaf7dc",
                                               redirect_uri="https://example.com/callback",
                                               scope='user-library-read user-top-read playlist-read-private user-read-recently-played'))


In [94]:
token = sp.auth_manager.get_access_token(as_dict=False)
token

'BQA4OHzTI0gkZWIEu0_HFiwacy4UpD1dCUH8JyIME4lU-CU_pMx4RSlC6RrUb6ijYS8D0U5nMo2Bigyilb3tM6zNhi_6K4X9DrP3mGCGKjnguWiJ3Z_kRQLfm-NBJjC7Xk555WonilYMBJ9xuoau6DUkAc_TfTCk7I-VfJ7Yn9oHoPpjr7n30ESs05aNhayhapggInpKUnYsuEyEcl9e-Ewjs90LpxiEinHqK7dXtrMhn452UQ_I_wVnKV7FuGSsM_KsoY9RguFohQ'

In [112]:
playlists = sp.current_user_playlists()

#Print playlist names and URLs
print("Your playlists:")
for playlist in playlists['items']:
    print(f"- {playlist['name']} ({playlist['external_urls']['spotify']})")

Your playlists:
- SofayGoat (https://open.spotify.com/playlist/3d9Tnhh1xqBjrNrEJzQPy4)
- Top 50 All-Time Hits (https://open.spotify.com/playlist/3z1NkCdPdePZFJghVIsh4X)
- Wrapped Almost (https://open.spotify.com/playlist/20YdXM5rT1bsH6k4f1q1cZ)
- RNB Songs (https://open.spotify.com/playlist/3Uie4roFo10l5K6xccBZNh)
- Top 50 All-Time Hits (https://open.spotify.com/playlist/2UyrUaHPGKv6U9ixaWYXV7)
- Tender Taylor Moments (https://open.spotify.com/playlist/4HLeuVUoGMRSswCWE02Nnq)
- Volleyball Vibes (https://open.spotify.com/playlist/1s7ltpuJH4HH0SNBwMclYJ)
- Top Picks for Your Aux (https://open.spotify.com/playlist/7FX22E5EalZmpMd71uigGK)
- Musical Surprises (https://open.spotify.com/playlist/37UyPGU6f37LXLz9CW5bH1)
- Frat House Bangers (https://open.spotify.com/playlist/2O73W4iiGliWfnrEFRKqTR)
- Stats Study Focus (https://open.spotify.com/playlist/6qFbf4HRIssBjQA8PCr47y)
- Chill Lofi Favorites (https://open.spotify.com/playlist/79Gd1i4gWzmr4TRRK4bZcu)
- Shower Vibes: Rap & Pop (https://op

In [114]:
recents = sp.current_user_recently_played(limit=10)

#Prints recent songs from user's Spotify
for item in recents['items']:
    track = item['track']
    played_at = item['played_at']
    print(f"{track['name']} by {track['artists'][0]['name']} (played at {played_at})")


WTHELLY by Rob49 (played at 2025-05-22T01:21:38.358Z)
LiveLeak by Ken Carson (played at 2025-05-22T01:19:21.209Z)
Hell Yeah by SoFaygo (played at 2025-05-22T01:16:09.196Z)
What Are You Waiting For by d4vd (played at 2025-05-22T01:13:20.806Z)
TRIM (with Future) by Playboi Carti (played at 2025-05-22T01:10:55.944Z)
Let's Lose Our Minds by SoFaygo (played at 2025-05-22T01:08:00.428Z)
Free Agents by Homixide Gang (played at 2025-05-22T01:01:36.073Z)
TIFFANY by Benji Blue Bills (played at 2025-05-22T00:58:14.154Z)
not yo gang by Glo (played at 2025-05-22T00:56:04.837Z)
promises by Hardrock (played at 2025-05-22T00:54:33.869Z)


In [116]:
top_tracks = sp.current_user_top_tracks(limit=5, time_range='short_term')
seed_ids = [track['id'] for track in top_tracks['items']]

print(seed_ids)
print(len(seed_ids))


['02CUgLutqmue2cayDgmn53', '6GcUFZ5PEAo5gc4IvfsGTv', '1LsisG6aoqfsF1r6M4LpMO', '468SJikdm8k5a9SeahqkC8', '4bYOJrnVH6wS39xKaY6UEn']
5


In [122]:
#This code block generates song recommendations purely off user's most replayed tracks
#Low level example, generates mainstream recommendations

#Step 1: Fetch your top tracks
top_tracks = sp.current_user_top_tracks(limit=20, time_range='short_term')
track_data = [{
    'name': track['name'],
    'artist': track['artists'][0]['name'],
    'id': track['id'],
    'popularity': track['popularity']
} for track in top_tracks['items']]

df_top_tracks = pd.DataFrame(track_data)

#Step 2: Analyze most frequent artists
top_artists = df_top_tracks['artist'].value_counts().head(3).index.tolist()

#Step 3: Search for similar tracks by top artists
recommendations = []
for artist in top_artists:
    results = sp.search(q=f'artist:{artist}', type='track', limit=5)
    for item in results['tracks']['items']:
        recommendations.append({
            'name': item['name'],
            'artist': item['artists'][0]['name'],
            'id': item['id'],
            'popularity': item['popularity']
        })

df_recommendations = pd.DataFrame(recommendations)

#Step 4: Display recommendations
print("Recommended Tracks:")
print(df_recommendations[['name', 'artist', 'popularity']])


Recommended Tracks:
                                                 name      artist  popularity
0   Off The Meter (with Playboi Carti & Destroy Lo...  Ken Carson          77
1                                            overseas  Ken Carson          76
2                                                  ss  Ken Carson          82
3                                  Fighting My Demons  Ken Carson          78
4                                                Yale  Ken Carson          80
5                                 Don't Hold The Fire     SoFaygo          63
6                                            Everyday     SoFaygo          59
7                                         Off The Map     SoFaygo          60
8                                           Hell Yeah     SoFaygo          60
9                                         Knock Knock     SoFaygo          69
10                                               Sing   556Kurumi          39
11                                          

In [126]:
artist_ids = [track['artists'][0]['id'] for track in top_tracks['items']]
artist_meta = sp.artists(artist_ids)

#Prints user's top artists and their genres based on most listened to songs recently
for artist in artist_meta['artists']:
    print(f"{artist['name']}: {artist['genres']}")

SoFaygo: ['rage rap']
nabh: []
Ken Carson: ['rage rap']
d4vd: []
Hardrock: ['rage rap', 'underground hip hop']
Ken Carson: ['rage rap']
Benji Blue Bills: []
Duwap Kaine: ['cloud rap', 'underground hip hop']
k3: []
Ken Carson: ['rage rap']
lil batou: []
Ken Carson: ['rage rap']
Ariana Grande: ['pop']
556Kurumi: []
Ken Carson: ['rage rap']
Ken Carson: ['rage rap']
Young Thug: ['rap', 'melodic rap', 'trap']
556Kurumi: []
Homixide Gang: ['rage rap']
SoFaygo: ['rage rap']


In [128]:
genre_counter = Counter()
for artist in artist_meta['artists']:
    genre_counter.update(artist['genres'])

#Top genres in user's profile
top_genres = genre_counter.most_common(5)
print("Top genres:", top_genres)


Top genres: [('rage rap', 10), ('underground hip hop', 2), ('cloud rap', 1), ('pop', 1), ('rap', 1)]


In [135]:
#Searches for songs based on top_genres and appends them to results
for genre, _ in top_genres:
    results = sp.search(q=f'genre:"{genre}"', type='track', limit=10) #can adjust this limit to change amt of generated songs

#Prints list of recommended songs
for item in results['tracks']['items']:
        track = item['name']
        artist = item['artists'][0]['name']
        print(f"  - {track} by {artist}")

  - No Pole by Don Toliver
  - All The Stars (with SZA) by Kendrick Lamar
  - Snooze by SZA
  - See You Again (feat. Kali Uchis) by Tyler, The Creator
  - HUMBLE. by Kendrick Lamar
  - Like That by Future
  - Not Like Us by Kendrick Lamar
  - Money Trees by Kendrick Lamar
  - The Spins by Mac Miller
  - I Had Some Help (Feat. Morgan Wallen) by Post Malone
